In [18]:
# Librerias
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error
import mlflow

In [19]:
# Funcion para leer los datos y transformarlos
def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

# Concatenar el ID de Pick Up y Dropoff
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

# Vectorizar los valores categoricos
dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

In [ ]:
# Definir donde iba a vivir el experiment tracking server
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Nombrar experimento
mlflow.set_experiment("class-nyc-taxi-experiment")

# Iniciar un run de MLflow
with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

# Model Registry Example

In [ ]:
# Librerias
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

In [ ]:
mlflow.sklearn.autolog()

In [ ]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [ ]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [ ]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            # !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

# Registrar modelo

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model" # Si no le pones esto, no te deja registrar el modelo
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

## Metodo 2 para registrar modelos

In [ ]:
# run_id = input("Ingrese el run_id")
# run_uri = f"runs:/{run_id}/model" # Aqui se loggea automanicamente con el ID del modelo

# result = mlflow.register_model(
#     model_uri=run_uri,
#     name="nyc-taxi-model"
# )

# Metodo 3 para registrar modelos

In [ ]:
from mlflow import MlflowClient
client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [ ]:
# client.create_registered_model(name="nyc-taxi-model") # Crear el modelo registrado, marca error pq ya tenemos

In [ ]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version( # Crea el modelo
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

# Asignar alias y descripciones

In [ ]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=7
)

In [23]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=5
)

# Actualizar una version de un modelo

In [24]:
client.update_model_version(
    name="nyc-taxi-model",
    version=5,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=['challenger'], creation_timestamp=1760668213899, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1760669266232, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='f1d84b7701df40e19bbf033d3ddcb0e7', run_link=None, source='models:/m-840e21536f2c4824b090dba320efed35', status='READY', status_message=None, tags={}, user_id=None, version=5>

# Borrar un alias

In [50]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

# Obtener version por alias

In [28]:
# get a model version by alias
champion_model = client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)
print(champion_model.version)

7


# Obtener modelo

In [ ]:
# por alias
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([15.02976639, 15.02976639, 15.02976639, ..., 15.02976639,
       15.02976639, 15.02976639], shape=(44218,))

In [37]:
# por version
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 7

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

array([15.02976639, 15.02976639, 15.02976639, ..., 15.02976639,
       15.02976639, 15.02976639], shape=(44218,))

# Comparar modelos

In [38]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [42]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")

In [43]:
# Descargar preprocesador
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'c:\\Users\\sarah\\apps\\sem_5\\pcd\\nyc-taxi-predictions-2025\\notebooks\\preprocessor'

In [44]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [45]:
X_test = preprocess(df, dv)

In [46]:
target = "duration"
y_test = df[target].values

In [55]:
test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)


{'rmse': 9.162768346025532}

In [56]:
test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)


{'rmse': 6.01147524270201}

In [ ]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=100
)
